In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
# Path to your service account key
service_account_path = 'atopile-880ca67acfe2.json'

# Initialize Firestore
if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    app = firebase_admin.initialize_app(cred)

db = firestore.client()
modules_collection = db.collection('packages')  # Replace 'modules' with your actual collection name


In [15]:
selected_types = ["regulator","buck"]
query_ref = modules_collection.where('types', 'array_contains_any', selected_types)

# put in a pandas dataframe
docs = query_ref.stream()
data = [doc.to_dict() for doc in docs]
df = pd.DataFrame(data)
df

/Users/narayanpowderly/Documents/atopile-workspace/.venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


,version,description,image,name,voltage_in_max,voltage_out_max,github-url,price,interfaces,voltage_in_min,types,current_max_A,voltage_out_min,current_range_A,voltage_in_range,voltage_out_range
0,0.1.0,Buck-Boost Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BB200,12.0,10.0,https://gitlab.atopile.io/packages/BB200,1.0,"[Power_in, Power_out]",3.0,"[buck-boost, regulator]",2.0,1.0,NaN,NaN,NaN
1,0.1.0,Adjustable Buck-Boost Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BB300,NaN,NaN,https://gitlab.atopile.io/packages/BB300,1.3,"[Power_in, Power_out]",NaN,"[buck-boost, regulator]",NaN,NaN,3,2 - 15,1.5 - 13
2,0.1.0,Buck Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BCK100,20.0,15.0,https://gitlab.atopile.io/packages/BCK100,0.7,"[Power_in, Power_out]",4.0,"[buck, regulator]",1.0,1.0,NaN,NaN,NaN
3,0.1.0,High Power Buck Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BCK500,30.0,20.0,https://gitlab.atopile.io/packages/BCK500,1.2,"[Power_in, Power_out]",5.0,"[buck, regulator]",5.0,3.0,NaN,NaN,NaN
4,0.1.0,Boost Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BST150,5.0,12.0,https://gitlab.atopile.io/packages/BST150,0.6,"[Power_in, Power_out]",0.9,"[boost, regulator]",1.5,5.0,NaN,NaN,NaN
5,0.1.0,High Efficiency Boost Converter,https://gitlab.atopile.io/uploads/-/system/pro...,BST250,4.0,25.0,https://gitlab.atopile.io/packages/BST250,0.8,"[Power_in, Power_out]",1.0,"[boost, regulator]",2.5,5.0,NaN,NaN,NaN
6,0.1.0,Flyback Converter,https://gitlab.atopile.io/uploads/-/system/pro...,FLY300,50.0,45.0,https://gitlab.atopile.io/packages/FLY300,1.5,"[Power_in, Power_out]",10.0,"[flyback, regulator]",3.0,5.0,NaN,NaN,NaN
7,0.1.0,LLC Converter,https://gitlab.atopile.io/uploads/-/system/pro...,LLC400,60.0,55.0,https://gitlab.atopile.io/packages/LLC400,2.0,"[Power_in, Power_out]",20.0,"[llc, regulator]",4.0,15.0,NaN,NaN,NaN
8,0.1.0,Linear Regulator,https://gitlab.atopile.io/uploads/-/system/pro...,LTR500,6.0,5.0,https://gitlab.atopile.io/packages/LTR500,0.3,"[Power_in, Power_out]",2.0,"[linear, regulator]",0.5,1.5,NaN,NaN,NaN
9,0.1.0,Advanced Linear Regulator,https://gitlab.atopile.io/uploads/-/system/pro...,LTR700,7.0,6.5,https://gitlab.atopile.io/packages/LTR700,0.5,"[Power_in, Power_out]",3.0,"[linear, regulator]",0.7,1.8,NaN,NaN,NaN


In [6]:
modules_table._read_table()

NameError: name 'modules_table' is not defined

In [15]:
import json

# Read the JSON file
with open('db.json') as f:
    data = json.load(f)

# stick it into a dataframe
df = pd.DataFrame(data['modules'])
df

JSONDecodeError: Expecting property name enclosed in double quotes: line 26 column 7 (char 577)

In [ ]:
def query_modules(selected_types, properties_filters):
    Module = Query()
    results = []

    # Search through all modules
    for module in modules_table.all():
        # Check if the module's type intersects with the selected types
        if set(module['type']).intersection(selected_types):
            # Further filter the module based on properties
            match = True
            for prop, value_range in properties_filters.items():
                # Assume each property filter defines min and max
                if 'min' in value_range and 'max' in value_range:
                    prop_min = float(module['properties'][prop]['min'].rstrip('V').rstrip('A'))  # Remove units for comparison
                    prop_max = float(module['properties'][prop]['max'].rstrip('V').rstrip('A'))  # Remove units for comparison
                    filter_min = float(value_range['min'])
                    filter_max = float(value_range['max'])
                    if not (filter_min <= prop_min and prop_max <= filter_max):
                        match = False
                        break
            if match:
                results.append(module)

    return results

# Sample types and properties_filters for testing
selected_types = ["Regulator", "AdjustableRegulator"]
properties_filters = {
    "power_in.voltage": {'min': '0.5', 'max': '5'},
    "power_out.voltage": {'min': '1', 'max': '6'}
}

# Query the database
results = query_modules(selected_types, properties_filters)

# Convert the results to a Pandas DataFrame for better display
df = pd.DataFrame(results)

df.head()  # Display the first few rows of the dataframe